In [2]:
import pandas as pd

file_path = './totaldata.csv'
data = pd.read_csv(file_path)

data.head()

,SUB_ID,DX_GROUP,AGE_AT_SCAN,SEX,FIQ,VIQ,PIQ,EYE_STATUS_AT_SCAN,HANDEDNESS_L,HANDEDNESS_R,FILE_ID
0,50003,1,24.45,1,124.0,128.0,115.0,2,0,1,Pitt_0050003_func_preproc.nii.gz
1,50004,1,19.09,1,113.0,108.0,117.0,2,0,1,Pitt_0050004_func_preproc.nii.gz
2,50005,1,13.73,2,119.0,117.0,118.0,2,0,1,Pitt_0050005_func_preproc.nii.gz
3,50006,1,13.37,1,109.0,99.0,119.0,2,1,0,Pitt_0050006_func_preproc.nii.gz
4,50007,1,17.78,1,110.0,106.0,112.0,2,0,1,Pitt_0050007_func_preproc.nii.gz


In [3]:
import pandas as pd
import os
import nibabel as nib
from nilearn.datasets import fetch_atlas_basc_multiscale_2015
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
import numpy as np
import joblib

df = data

basc_atlas = fetch_atlas_basc_multiscale_2015(version='asym')
atlas_filename = basc_atlas['scale122']
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True)

time_series_list = []
labels = []

path =  'd:/ABIDE Analysis/nofilt_noglobal/' 

for idx, row in df.iterrows():
    file_name = row['FILE_ID']
    file_path = path + file_name
    if os.path.exists(file_path):
        nii_img = nib.load(file_path)
        time_series = masker.fit_transform(nii_img)
        time_series_list.append(time_series)
        labels.append(row['DX_GROUP'])

labels = np.array(labels)

print("Number of subjects:", len(time_series_list))
print("Labels shape:", labels.shape)

joblib.dump(time_series_list, 'time_series_list_122.pkl')
joblib.dump(labels, 'labels.pkl')

Number of subjects: 587
Labels shape: (587,)


['labels.pkl']

In [4]:
def extract_measure(type):
    conn_measure = ConnectivityMeasure(kind=type)
    connectivity_matrices = conn_measure.fit_transform(time_series_list)

    def flatten_lower_triangle(matrix):
        return matrix[np.tril_indices_from(matrix, k=-1)]
    
    return np.array([flatten_lower_triangle(matrix) for matrix in connectivity_matrices])

tangent = extract_measure('tangent')
correlation = extract_measure('correlation')
partial_correlation = extract_measure('partial correlation')
covariance =  extract_measure('covariance')

joblib.dump(correlation, 'correlation122.pkl')
joblib.dump(partial_correlation, 'partial_correlation122.pkl')
joblib.dump(covariance, 'covariance122.pkl')
joblib.dump(tangent, 'tangent122.pkl')

['tangent122.pkl']

In [5]:
totaldata_cov = pd.concat([data, pd.DataFrame(tangent)], axis=1)
totaldata_corr = pd.concat([data, pd.DataFrame(correlation)], axis=1)
totaldata_pcorr = pd.concat([data, pd.DataFrame(partial_correlation)], axis=1)
totaldata_tangent = pd.concat([data, pd.DataFrame(tangent)], axis=1)

In [6]:
totaldata_corr.to_csv('totaldata_correlation122.csv', index=False)
totaldata_pcorr.to_csv('totaldata_partialcorr122.csv', index=False)
totaldata_cov.to_csv('totaldata_covariance122.csv', index=False)
totaldata_tangent.to_csv('totaldata_tangent122.csv', index=False)

In [7]:
totaldata_tangent

,SUB_ID,DX_GROUP,AGE_AT_SCAN,SEX,FIQ,VIQ,PIQ,EYE_STATUS_AT_SCAN,HANDEDNESS_L,HANDEDNESS_R,...,7371,7372,7373,7374,7375,7376,7377,7378,7379,7380
0,50003,1,24.45,1,124.0,128.0,115.0,2,0,1,...,-0.077840,-0.050443,0.032142,0.112209,-0.012648,0.125946,-0.136083,0.198519,-0.130713,0.065370
1,50004,1,19.09,1,113.0,108.0,117.0,2,0,1,...,0.016702,-0.033594,0.049796,0.144990,-0.017183,0.036735,-0.108421,-0.033709,-0.066950,0.046501
2,50005,1,13.73,2,119.0,117.0,118.0,2,0,1,...,-0.188773,0.017362,0.100259,0.037980,-0.027972,0.035333,-0.061511,-0.104224,0.101761,-0.009351
3,50006,1,13.37,1,109.0,99.0,119.0,2,1,0,...,0.043947,0.006421,0.162764,0.010855,-0.023369,0.062912,0.022271,-0.150709,-0.015298,-0.054761
4,50007,1,17.78,1,110.0,106.0,112.0,2,0,1,...,0.071665,-0.050112,0.002150,0.042927,-0.173567,0.168069,0.053380,0.058754,-0.119321,-0.097573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,51491,2,56.20,1,120.0,109.0,128.0,2,0,1,...,-0.071006,-0.045016,0.088477,-0.116710,-0.158622,-0.168682,0.265776,-0.114320,-0.177980,-0.038089
583,51493,2,29.20,2,102.0,101.0,103.0,2,0,1,...,-0.051032,0.002497,-0.032207,-0.029778,-0.023136,0.058905,0.055829,-0.235713,-0.142211,0.010806
584,51578,1,33.00,1,125.0,129.0,113.0,2,0,0,...,0.015811,-0.141982,0.012553,-0.007700,-0.145973,0.150548,0.002133,-0.024623,0.195326,0.148949
585,51583,1,35.00,1,95.0,105.0,84.0,2,0,0,...,-0.083652,-0.099759,-0.165624,0.104928,-0.036203,-0.033180,-0.067884,-0.084913,0.123487,0.029766


In [42]:
print(partial_correlation.shape, tangent.shape, covariance.shape, correlation.shape)

(587, 19306) (587, 19306) (587, 19306) (587, 19306)
